In [ ]:
# code to mount my drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install the necessary requirements

!pip install -r '/content/drive/My Drive/My_Software_Projects/BLIP/requirements.txt'

**This section is for running an inference instant of BLIP**



In [ ]:
%cd /content/drive/MyDrive/My_Software_Projects/BLIP
!pip install -Uqq ipdb
import ipdb

In [ ]:
%pdb on

**Code to load images and preprocess them**

In [5]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def loadImage(imageSize, device, rawImage):

    # imgUrl = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
    # rawImage = Image.open(requests.get(imgUrl, stream=True).raw).convert('RGB')
    # imgUrl = '/content/drive/My Drive/My_Software_Projects/Input_Frames/time0_frame1.jpg'
    # rawImage = Image.open(imgUrl).convert('RGB')
    # rawImage = cv2.imread(imgUrl)
    rawImage = rawImage
    rawImage = cv2.cvtColor(rawImage, cv2.COLOR_BGR2RGB)
    rawImage = Image.fromarray(rawImage)


    w,h = rawImage.size
    display(rawImage.resize((w//5,h//5)))
    #cv2.imshow('image', raw_image)

    transform = transforms.Compose([
        transforms.Resize((imageSize,imageSize),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ])
    image = transform(rawImage).unsqueeze(0).to(device)

    return image

**Make directories utli function**

In [6]:
import os

def makeDirectory(path):

    try:

        os.makedirs(path, exist_ok=True)
        print("Directory '%s' created successfully" % path)

    except OSError as error:

        print("Directory already exist")
        pass

**Code to run an inference on the BLIP image captioning network from a video file**

In [ ]:
from models.blip import blip_decoder
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import cv2
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
currentTime = time.strftime("%H_%M_%S")

#path to input Video File
pathToInputVideoFile = '/content/drive/My Drive/My_Software_Projects/Input_Video/InputVideo2.mp4'

#path to the output Folder
pathToOutputFramesFolder = '/content/drive/My Drive/My_Software_Projects/Output/Output_Caption_' + str(currentTime) + '/'
makeDirectory(pathToOutputFramesFolder)

# path to the output text File
pathToOutputTxtFile = '/content/drive/My Drive/My_Software_Projects/Output_File/CaptionTextFile_' + str(currentTime) + '.txt'

# set image size
imageSize = 384

#Loading the model file
modelUrl = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'



model = blip_decoder(pretrained=modelUrl, image_size=imageSize, vit='base')
model.eval()
model = model.to(device)
print("Model Successfully Loaded.")

print("Loading Input Video File.....")
inputVideo = cv2.VideoCapture(pathToInputVideoFile)
print("Successfully Loaded input File")


# Calculate the Frames per second (FPS)
print("Calculating Frames Per Second...")
fps = round(inputVideo.get(cv2.CAP_PROP_FPS))
print('Fps = ' + str(fps))

frameNumber = 0
timeStamp = 0

print('Processing Frames...')


while True:
  # Processing Frames
  success, imageFrame = inputVideo.read()

  if success:
    # increase the frame by 1
    frameNumber += 1

    image = loadImage(imageSize=imageSize, device=device, rawImage=imageFrame)

    with torch.no_grad():

      # ipdb.set_trace(context=6)
      # beam search
      caption = model.generate(image, sample=False, num_beams=3, max_length=20, min_length=5)

      # nucleus sampling
      #caption = model.generate(image, sample=True, top_p=0.9, max_length=20, min_length=5)

      print('caption: '+caption[0])

      # outputFrameFilePath = pathToOutputFramesFolder + 'time' + str(timeStamp) + '_' + 'frame' + str(frameNumber) + '.jpg'

      outputFrameFilePath = pathToOutputFramesFolder + str(caption[0]) + '.jpg'

      # write the frame
      cv2.imwrite(outputFrameFilePath, imageFrame)

      # write the captions to an output file
      writingText = "TimeStamp = " + str(timeStamp) +  " Frame = " + str(frameNumber) + " caption : " + str(caption[0])

      with open(pathToOutputTxtFile, 'a') as testwritefile:
        testwritefile.write(writingText + '\n')

      print('Time = ' + str(timeStamp) + ' secs Frame = ' + str(frameNumber) + ' saved successfully')

  else:

    break

  # every 24 Frames increase the time by 1 and reset the frames to 0
  if frameNumber == fps:

    timeStamp += 1
    frameNumber = 0

print("Frame successfully Processed.")
inputVideo.release()

**Inference code for VQA (Visual Question Answering)**

In [ ]:
from models.blip_vqa import blip_vqa
from models.blip import blip_decoder
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import cv2
import time

# question to be Asked (Text Prompt)
question = 'What is the facial expression of the person?'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
currentTime = time.strftime("%H_%M_%S")

#path to input Video File
pathToInputVideoFile = '/content/drive/My Drive/My_Software_Projects/Input_Video/InputVideo2.mp4'

#path to the output Folder
pathToOutputFramesFolder = '/content/drive/My Drive/My_Software_Projects/Output/Output_VQA_' + str(currentTime) + '/'
makeDirectory(pathToOutputFramesFolder)

# path to the output text File
pathToOutputTxtFile = '/content/drive/My Drive/My_Software_Projects/Output_File/CaptionTextFile_' + str(currentTime) + '.txt'

imageSize = 480
# image = load_image(image_size=image_size, device=device)

modelUrl = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'

print("Loading the Pre-Trained Model...")

model = blip_vqa(pretrained=modelUrl, image_size=imageSize, vit='base')
model.eval()
model = model.to(device)
print("Model Successfully Loaded.")

print("Loading Input Video File.....")
inputVideo = cv2.VideoCapture(pathToInputVideoFile)
print("Successfully Loaded input File")


# Calculate the Frames per second (FPS)
print("Calculating Frames Per Second...")
fps = round(inputVideo.get(cv2.CAP_PROP_FPS))
print('Fps = ' + str(fps))

frameNumber = 0
timeStamp = 0

print('Processing Frames...')


while True:
  # Processing Frames
  success, imageFrame = inputVideo.read()

  if success:
    # increase the frame by 1
    frameNumber += 1

    image = loadImage(imageSize=imageSize, device=device, rawImage=imageFrame)

    with torch.no_grad():

      answer = model(image, question, train=False, inference='generate')
      print('answer: '+answer[0])


      outputFrameFilePath = pathToOutputFramesFolder + str(answer[0]) + str(timeStamp) + str(frameNumber) + '.jpg'

      # write the frame
      cv2.imwrite(outputFrameFilePath, imageFrame)

      # write the captions
      writingText = "TimeStamp = " + str(timeStamp) +  " Frame = " + str(frameNumber) + " Emotion : " + str(answer[0])
      # utputTextFile.write(writingText)

      with open(pathToOutputTxtFile, 'a') as testwritefile:
        testwritefile.write(writingText + '\n')

      # outputTextFile.write('\n')

      print('Time = ' + str(timeStamp) + ' secs Frame = ' + str(frameNumber) + ' saved successfully')

  else:

    break

  # every 24 Frames increase the time by 1 and reset the frames to 0
  if frameNumber == fps:

    timeStamp += 1
    frameNumber = 0

print("Frame successfully Processed.")
inputVideo.release()

